# 2023微博热搜分析

In [1]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.charts import WordCloud

### 将数据保存在文本文档中，查看其中的前二十条

In [2]:
f2 = open("new.txt", "r",encoding='utf-8')
lines = f2.readlines()
df = pd.DataFrame(columns=("date", "title", "searchCount"))
a = 0
for line in lines:
    csvRow = line.split(",")
    s = {}
    s["date"] = str(csvRow[0])
    s["title"] = str(csvRow[1])
    s["searchCount"] = int(csvRow[2])
    df.loc[a] = s
    a += 1

print(df.head(20))

        date            title  searchCount
0   20230101         王俊凯单手吊威亚     13069032
1   20230101          王一博赤脚跳舞     12468653
2   20230101            虞书欣 妆     10680913
3   20230101         赵露思腮红谁打的      8226204
4   20230101             五个王源      5774735
5   20230101          鞠婧祎胎毛刘海      5584814
6   20230101              xbb      5028072
7   20230101             诺氟沙星      3452889
8   20230101    摩洛哥禁止来自中国旅客入境      3237618
9   20230101   外来媳妇本地郎康伯扮演者去世      2373537
10  20230101           小破站 整活      1702510
11  20230101            程潇 假唱      5523801
12  20230101             何炅哭了      3317652
13  20230101        苏醒说哥是懂站位的      1864616
14  20230101            刘亦菲酒量      1620378
15  20230101        王毅任中央外办主任      1294525
16  20230101     祝愿大家新年快乐皆得所愿      5981648
17  20230101  南部战区揭露美军机抵近侦察真相      5956720
18  20230101       大家都在坚忍不拔努力      4847272
19  20230101           周深跨年好忙      3660220


## 1.热搜分析

In [3]:
df1 = df.sort_values(by='searchCount',ascending=False)
df1

,date,title,searchCount
31471,20231030,李克强同志逝世,133098325
2922,20230201,胡歌 我当爸爸啦,71434818
36438,20231219,甘肃地震,63844263
17375,20230706,李玟去世,60048669
17393,20230706,李玟抑郁症,45339693
...,...,...,...
8942,20230406,编剧吴迪,500039
33326,20231118,你好星期六下周有马丽赵露思,500033
27562,20230924,田嘉瑞有工作室了,500024
36529,20231219,刘宇宁 换脸,500019


#### 选排名前50个热搜话题,并做成可视化图表

In [4]:
df2 = df1.head(500)
df2

,date,title,searchCount
31471,20231030,李克强同志逝世,133098325
2922,20230201,胡歌 我当爸爸啦,71434818
36438,20231219,甘肃地震,63844263
17375,20230706,李玟去世,60048669
17393,20230706,李玟抑郁症,45339693
...,...,...,...
31172,20231027,花16万买了辆二手奔驰S,5216073
24620,20230830,易建联退役,5211917
2702,20230130,张公案,5210000
14876,20230617,上海地震,5202107


In [5]:
%matplotlib inline

In [6]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'  # 或者 'png'

In [7]:
c = (
    Bar()
    .add_xaxis(df2['title'].tolist())
    .add_yaxis('搜索量', df2['searchCount'].tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))

    .set_global_opts(
        brush_opts=opts.BrushOpts(),
        title_opts=opts.TitleOpts(title="  ", subtitle="搜索量"),
        datazoom_opts=[opts.DataZoomOpts(range_start=0,range_end=10)]
    )
    #.render("bar_with_brush.html")
    .render_notebook()
)
c



为了更直观的展现，网友对哪些话题搜索量更高，接下来将这些话题绘制成词云。

In [8]:
words = df2['title'].tolist()
frequency = df2['searchCount'].tolist()
data = zip(words,frequency)

In [9]:
wordcloud = (
    WordCloud()
    .add(series_name="话题搜索频率", data_pair=data, word_size_range=[6, 46])
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="话题搜索频率", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
    #.render("word-cloud.html")
    .render_notebook()
)
wordcloud

结合微博热搜话题搜索，哪些热搜的热度高一目了然，在榜单上可以看出全年热搜搜索量第一的是李克强同志逝世的消息，从数据上来看是第二名的将近两倍，由此见得网民对国家领导人的情况还是尤为关注；在这榜单上可以看出23年不算太平，有着不少的自然灾害的发生以及国际形势的不稳定，这些消息也占据了许多热搜；许多明星的“大事”也位居前列，例如“胡歌当爸爸”、“易建联退役”等等

## 2.热词分析

#### 想看看23年哪些明星是热搜的宠儿。
#### 需要用到的库是jieba

In [10]:
import jieba

In [11]:
title_list = df1['title'].tolist()

In [12]:
for i in title_list[0:50]:
    data_cut=jieba.cut(i,use_paddle=True)
    print(list(data_cut))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\96213\AppData\Local\Temp\jieba.cache
Loading model cost 0.654 seconds.
Prefix dict has been built successfully.


['李克强', '同志', '逝世']
['胡歌', ' ', '我', '当', '爸爸', '啦']
['甘肃', '地震']
['李玟', '去世']
['李玟', '抑郁症']
['快', '告诉', '我', '是', '假', '的']
['蔡', '徐坤', ' ', '照片', '不是', '我']
['蔡', '徐坤', '回应']
['蔡', '徐坤', '工作室', '声明']
['周海媚', '去世']
['蔡', '徐坤', '否认', '女方', '未成年', '及', '强制', '堕胎']
['王毅任', '外交部', '部长']
['山东', '地震']
['泸定', '地震']
['李佳琦哭', '着', '道歉']
['再见', '周芷若']
['汪峰发', '了', '个', '锤子']
['北京', '震感']
['沈腾', '吃药']
['日本', '核污染', '水排', '海', '正式', '开始']
['T1', '零封', 'WBG']
['TFBOYS', '十年', '之约', '演唱会']
['秦刚', '外长', '职务', '被', '免去']
['高考作文']
['胡鑫宇', '遗体', '被', '发现']
['十周年', '蛋糕', '塌', '了']
['日本', '核污染', '水', '下午', '排入', '海洋']
['高考', '语文']
['羽生', '结弦', '宣布', '结婚']
['张继科', '借条']
['习近平', '当选', '中华人民共和国', '主席']
['仲尼']
['官方', '通报', '北极', '鲶鱼', '事件']
['WBG', '对战', 'T1']
['TFBOYS', '官博急', '了']
['高考', '数学']
['QQ', '崩', '了']
['甘肃', '地震', '已致', '111', '人', '遇难']
['甘肃', '地震', '已致', '86', '人', '遇难', '96', '人', '受伤']
['邱耀乐', '不', '接受', '炎亚纶', '道歉']
['黄子佼', '曝大', '小S', '吸毒']
['微博之夜', '世纪', '同框']
['T1', '晋级', 'S13', '总决赛']
['胡

#### 从得到的结果不难看出，在拆分的过程中，‘蔡徐坤’被拆成了‘蔡’+‘徐坤’，‘汪峰发了个锤子’给拆成了‘汪峰发’
#### 为了解决这个问题，需要创造一个词库，将艺人的名字放到词库中，以此提高分词的准确性

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time

In [26]:
browser = webdriver.Chrome()
browser.implicitly_wait(3)
browser.get('https://www.baidu.com/s?wd=%E4%B8%AD%E5%9B%BD%E5%90%8D%E6%98%9F%E5%A4%A7%E5%85%A8&rsv_spt=1&rsv_iqid=0xee40b2550008d810&issp=1&f=8&rsv_bp=1&rsv_idx=2&ie=utf-8&rqlang=cn&tn=85070231_49_hao_pg&rsv_enter=1&rsv_dl=tb&oq=%25E4%25B8%25AD%25E5%259B%25BD%25E5%2590%258D%25E4%25BA%25BA%25E5%25A4%25A7%25E5%2585%25A8&rsv_t=0916iu0zHmwpTD1ziV6HB4WHQimT9qPArBWs7WSeQffmrx4h6tSZxbzmSoDFxrRhGuY5b9LpuplC&rsv_btype=t&inputT=7610&rsv_pq=d1826ff400093ac8&rsv_sug3=59&rsv_sug1=61&rsv_sug7=100&rsv_sug2=0&rsv_sug4=12633')
#将页面上的明星名字全都拿下来
china_star_names = set()
page = 0

while True:
    if page < 100:
        time.sleep(2)
        name_elements = browser.find_elements(By.CSS_SELECTOR, '#carousel > div._container_1so0w_1._container_1ktf7_1 > div div > h3 > a > div > div > p > span > span')
        for element in name_elements:
            temp = element.get_attribute('textContent')
            china_star_names.add(temp)

        next_button = browser.find_element(By.CSS_SELECTOR, 'div > div._pagination-button_1e5eg_1._pagination-next-button_1e5eg_43')
        if 'disabled' in next_button.get_attribute('class'):
            break
        else:
            next_button.click()
    else:
        break
    page += 1


#名字数量
print('名人名字数量为：')
print(len(china_star_names))

名人名字数量为：
908


In [27]:
with open('china_star_names.txt', 'w') as file:
    for name in china_star_names:
        file.write(name + '\n')
# 打印出名字的数量
print("txt文件中有{}个名字".format(len(china_star_names)))

txt文件中有908个名字


##### （蔡徐坤因为劣迹艺人已百度从中国明星中除名，为了分析只能手动添加了）

In [28]:
for name in china_star_name:
    jieba.add_word(name)
jieba.add_word('蔡徐坤')

In [29]:
cut_word_list = []
for i in title_list:
    data_cut=jieba.cut(i,use_paddle=True)
    cut_word_list.append(list(data_cut))

In [30]:
cut_word = []
for i in cut_word_list:
    for j in i:
        cut_word.append(j)
        
# 分词结果
cut_word

['李克强',
 '同志',
 '逝世',
 '胡歌',
 ' ',
 '我',
 '当',
 '爸爸',
 '啦',
 '甘肃',
 '地震',
 '李玟',
 '去世',
 '李玟',
 '抑郁症',
 '快',
 '告诉',
 '我',
 '是',
 '假',
 '的',
 '蔡徐坤',
 ' ',
 '照片',
 '不是',
 '我',
 '蔡徐坤',
 '回应',
 '蔡徐坤',
 '工作室',
 '声明',
 '周海媚',
 '去世',
 '蔡徐坤',
 '否认',
 '女方',
 '未成年',
 '及',
 '强制',
 '堕胎',
 '王毅任',
 '外交部',
 '部长',
 '山东',
 '地震',
 '泸定',
 '地震',
 '李佳琦哭',
 '着',
 '道歉',
 '再见',
 '周芷若',
 '汪峰',
 '发了',
 '个',
 '锤子',
 '北京',
 '震感',
 '沈腾',
 '吃药',
 '日本',
 '核污染',
 '水排',
 '海',
 '正式',
 '开始',
 'T1',
 '零封',
 'WBG',
 'TFBOYS',
 '十年',
 '之约',
 '演唱会',
 '秦刚',
 '外长',
 '职务',
 '被',
 '免去',
 '高考作文',
 '胡鑫宇',
 '遗体',
 '被',
 '发现',
 '十周年',
 '蛋糕',
 '塌',
 '了',
 '日本',
 '核污染',
 '水',
 '下午',
 '排入',
 '海洋',
 '高考',
 '语文',
 '羽生',
 '结弦',
 '宣布',
 '结婚',
 '张继科',
 '借条',
 '习近平',
 '当选',
 '中华人民共和国',
 '主席',
 '仲尼',
 '官方',
 '通报',
 '北极',
 '鲶鱼',
 '事件',
 'WBG',
 '对战',
 'T1',
 'TFBOYS',
 '官博急',
 '了',
 '高考',
 '数学',
 'QQ',
 '崩',
 '了',
 '甘肃',
 '地震',
 '已致',
 '111',
 '人',
 '遇难',
 '甘肃',
 '地震',
 '已致',
 '86',
 '人',
 '遇难',
 '96',
 '人',
 '受伤',
 '邱耀乐',
 '不',
 '接受',
 '炎亚纶'

### 再将获得的数据写入jieba的词库中，再做一次分词。可以看出此时的效果已经比较好了，蔡徐坤，汪峰，彭昱畅都没有再被切开。不过仍然有一些分词的精准度的问题无法被结局，比如“中国好声音” 这个词会被分为“中国” “好” “声音”；“奔跑吧”会被分成“奔跑”“吧”。但是因为这里是想做“哪些明星人物经常出现在热搜”的研究，所以这部分的误差暂时不在考虑范围内。解决的方法可能是去寻找更符合社交媒体语言习惯的词库。 

### 接下来尝试初步统计一下每个词出现的频率：

In [31]:
from collections import Counter
# 计算每个（分词）后的词在热搜种出现的频率
arr=Counter(cut_word)
print(arr) 

Counter({'的': 4978, '了': 3557, ' ': 3287, '被': 2431, '是': 1538, '在': 1140, '回应': 1035, '我': 945, '中国': 880, '岁': 878, '人': 835, '女子': 794, '不': 771, '男子': 684, '说': 679, '后': 671, '和': 638, '有': 626, '你': 595, '一': 555, '吗': 544, '将': 531, '年': 528, '3': 526, '称': 520, '万': 506, '都': 477, '2': 474, '给': 465, '上': 441, '演唱会': 435, '为': 433, '到': 404, '1': 399, '已': 369, '4': 349, '好': 343, '元': 341, '个': 339, '自己': 321, '看': 320, '吃': 319, '与': 317, '女生': 316, '去': 314, '就': 308, '要': 300, '女孩': 298, '迪丽热巴': 296, '10': 294, '为什么': 294, '5': 293, '把': 289, '也': 288, '没': 288, '肖战': 287, '什么': 280, '发现': 275, '女儿': 275, '多': 274, '月': 274, '日本': 273, '妈妈': 270, '6': 265, '中': 263, '还': 258, '韩国': 258, '对': 250, '让': 250, '小': 245, '孩子': 243, '杨紫': 241, '用': 238, '真的': 236, '去世': 235, '一个': 235, '大': 234, '杭州': 234, '买': 234, '会': 229, '公司': 222, '直播': 221, '很': 217, '拍': 215, '2023': 210, '做': 210, '8': 209, '通报': 208, '这': 208, '没有': 206, '上海': 203, '警方': 203, '来': 201, '又': 200, '吧': 19

### 可以看出有很多无意义的词语，例如：“的”、“了”、“是”等等。为了删除这些无意义的词语，我引用了github上的词库：https://github.com/elephantnose/characters
### 这个停用词库是中文停用词表, 哈工大停用词表, 百度停用词表, 四川大学机器智能实验室停用词库四份停用词表进行了合并去重得到的，仔细研究后，发现这个停用词库中包含了一些词语，例如：“结果”、“满足”。这些词语在做大段文章的文本分析中，可以被视为无意义的。但微博热搜由于词语短且少，若将这些词都删去，则会影响这次分析的项目，因此我只选取了前382个词（到“一”这个字）。

#### 逐行读stopword_china.txt里的文本

In [32]:
stopword_txt = []
for line in open(r'./stop_words_china.txt',encoding = 'utf-8'):   
    stopword_txt.append(line.replace('\n','')) 
print(stopword_txt)

['"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', ']', '_', '}', '·', '×', 'Δ', 'Ψ', 'γ', 'μ', 'φ', 'В', '—', '‘', '’', '“', '”', '℃', 'Ⅲ', '↑', '→', '≈', '①', '②', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '■', '▲', '、', '。', '〉', '《', '》', '」', '『', '』', '【', '】', '〔', '〕', '㈧', '一', '上', '下', '不', '与', '且', '个', '临', '为', '乃', '么', '之', '乎', '乘', '也', '了', '于', '些', '亦', '人', '今', '仍', '从', '他', '以', '们', '任', '会', '但', '何', '你', '使', '依', '俺', '倘', '借', '像', '儿', '兮', '其', '内', '再', '冒', '冲', '几', '凡', '凭', '则', '别', '到', '即', '却', '去', '又', '及', '另', '只', '叫', '可', '各', '同', '后', '向', '吓', '吗', '吧', '吱', '呀', '呃', '呕', '呗', '呜', '呢', '呵', '呸', '咋', '和', '咚', '咦', '咧', '咱', '咳', '哇', '哈', '哉', '哎', '哗', '哟', '哦', '哩', '哪', '哼', '唉', '啊', '啐', '啥', '啦', '喂', '喏', '喽', '嗡', '嗬', '嗯', '嗳', '嘎', '嘘', '嘛', '嘻', '嘿', '因', '在', '地', '多', '大', '她', '好', '如', '宁', '它', '对', '将', '小', '尔',

#### 还要加上空格在这个stopword列表里才比较完整

In [21]:
stopword_txt.append(' ')

clean_cut_word=[]
for word in cut_word:
    if word not in stopword_txt:
        clean_cut_word.append(word)
    else:
        #看看哪些词被删掉了
        print(word)

 
我
当
啦
我
是
的
 
不是
我
及
着
个
开始
被
被
了
宣布
了
了
人
人
人
不
在
被
我
了
被
也
必然
被
一起
经
首先
在
8
 
一
出来
就
的
 
好
是
的
吧
 
被
吧
 
一
 
到底
不
如
 
被
被
 
在
被
多
是
 
大家
的
与
和
 
以后
会
更加
严格
自己
要求
再
 
再
与
给
到
vs
是
谁
在
 
for
 
you
在
将
处理
又
上
不
了
某某
被
3
与
好
 
对
 
了
被
 
4
在
 
于
和
了
被
vs
4
后
去
哪
了
应该
被
 
我
了
 
你
2
在
7
个
 
被
采取
一起
某某
被
了
将
的
将
向
好
vs
的
被
仍
已
被
为
被
被
好
到底
了
什么
被
被
已
已
再
被
是
最
的
1
 
会
这
吧
等到
 
在
去
哪
了
是
被
 
不是
不是
任
 
还是
好
当
他
从
 
了
都
在
被
没有
vs
 
一
你
的
大
人
 
好
被
后
联系
他
 
了
2
吧
相信
 
你
谁
打
的
人
 
我
是
个
的
人
被
采取
为
一
1
人
 
我
是
 
自己
 
及
已
完成
一
5
于
和
是
被
 
并
好
的
被
不
没有
至今
已
被
2
的
了
的
他们
在
有
哪些
遇到
应该
如何
处理
又
去
了
我
的
下
用
也
这么
的
一次
又
一个
出现
了
下
一个
还是
因
与
2
的
最后
一天
还
被
谁
了
不
的
人
每天
多
只有
的
来
大家
得
最
的
是
什么
人
完成
第
将
的
如何
不
看
独自
安全
的
了
被
是
是
了
正常
了
有
只
的
向
后
个人
了
2
在
的
了
1
6
8
种
的
有
后
被
了
的
看
上
了
了
为何
能
大
被
就
被
一起
故意
致
6
1
 
多次
冲
 
我
的
来
了
 
被
因
被
为
 
的
到底
是
的
让
人
了
 
的
都
做到
了
将
其
 
被
被
你
别
被
采取
今天
是
第
个
不是
的
好
 
是
和
谁
的
 
重要
吗
1
加入
 
 
上
只有
我
自己
了
让
再
一次
3
比
4
各地
他
认真
的
了
℃
到
了
结果
从
1
将
种
的
给
突然
的
了
 
 
不
大家
为什么
这么
我
全面
你

#### 现在再次对分词们做一次词频统计：

In [22]:
arr=Counter(clean_cut_word)
print(arr) 

Counter({'回应': 1035, '中国': 880, '岁': 878, '女子': 794, '男子': 684, '说': 680, '年': 567, '称': 520, '万': 508, '演唱会': 435, '元': 341, '吃': 319, '女生': 316, '女孩': 298, '迪丽热巴': 296, '10': 294, '没': 290, '肖战': 288, '发现': 275, '女儿': 275, '月': 275, '日本': 273, '中': 271, '妈妈': 270, '韩国': 258, '孩子': 243, '杨紫': 241, '真的': 236, '买': 236, '去世': 235, '杭州': 234, '公司': 222, '直播': 221, '拍': 220, '做': 212, '2023': 210, '通报': 208, '新': 207, '上海': 203, '警方': 203, '事件': 198, '杨幂': 196, '穿': 196, '视频': 195, '天': 194, '易烊千玺': 192, '学生': 190, '魏大勋': 189, '里': 187, '20': 186, '儿子': 185, '地震': 184, '官宣': 184, '棣': 183, '时': 182, '王一博': 181, '网友': 180, '北京': 179, '女': 179, 'TFBOYS': 178, '结婚': 177, '小时': 175, '王鹤': 172, '带': 172, '手机': 172, '曝': 169, '高考': 168, '花': 167, '老人': 167, '名': 166, '男孩': 165, '道歉': 163, '赵丽颖': 163, '长': 161, '想': 159, '父亲': 158, '老师': 157, '建议': 156, '造型': 156, '大学生': 156, '遭': 155, '张': 151, '前': 151, '死': 148, '哭': 146, '亿': 146, '官方': 143, '丫丫': 141, '30': 141, '死亡': 140, '工作': 139, '原因': 

#### 将统计的数据建成字典

In [23]:
data2 = dict(arr)
data2

{'李克强': 11,
 '同志': 13,
 '逝世': 15,
 '胡歌': 53,
 '爸爸': 132,
 '甘肃': 40,
 '地震': 184,
 '李玟': 87,
 '去世': 235,
 '抑郁症': 32,
 '快': 44,
 '告诉': 24,
 '假': 24,
 '蔡徐坤': 77,
 '照片': 73,
 '回应': 1035,
 '工作室': 106,
 '声明': 45,
 '周海媚': 25,
 '否认': 72,
 '女方': 14,
 '未成年': 18,
 '强制': 21,
 '堕胎': 2,
 '王毅任': 2,
 '外交部': 22,
 '部长': 11,
 '山东': 86,
 '泸定': 3,
 '李佳琦哭': 3,
 '道歉': 163,
 '再见': 24,
 '周芷若': 4,
 '汪峰': 5,
 '发了': 28,
 '锤子': 2,
 '北京': 179,
 '震感': 20,
 '沈腾': 59,
 '吃药': 2,
 '日本': 273,
 '核污染': 46,
 '水排': 20,
 '海': 28,
 '正式': 76,
 'T1': 34,
 '零封': 16,
 'WBG': 74,
 'TFBOYS': 178,
 '十年': 57,
 '之约': 11,
 '演唱会': 435,
 '秦刚': 3,
 '外长': 5,
 '职务': 7,
 '免去': 6,
 '高考作文': 5,
 '胡鑫宇': 68,
 '遗体': 43,
 '发现': 275,
 '十周年': 37,
 '蛋糕': 21,
 '塌': 7,
 '水': 54,
 '下午': 12,
 '排入': 1,
 '海洋': 24,
 '高考': 168,
 '语文': 2,
 '羽生': 15,
 '结弦': 19,
 '结婚': 177,
 '张继科': 29,
 '借条': 2,
 '习近平': 4,
 '当选': 18,
 '中华人民共和国': 3,
 '主席': 12,
 '仲尼': 13,
 '官方': 143,
 '通报': 208,
 '北极': 20,
 '鲶鱼': 20,
 '事件': 198,
 '对战': 49,
 '官博急': 1,
 '数学': 15,
 'QQ': 14,
 '崩': 29,


#### 将字典从大到小排序

In [24]:
data_sort = sorted(data2.items(), key=lambda x: x[1],reverse = True)
data_sort

[('回应', 1035),
 ('中国', 880),
 ('岁', 878),
 ('女子', 794),
 ('男子', 684),
 ('说', 680),
 ('年', 567),
 ('称', 520),
 ('万', 508),
 ('演唱会', 435),
 ('元', 341),
 ('吃', 319),
 ('女生', 316),
 ('女孩', 298),
 ('迪丽热巴', 296),
 ('10', 294),
 ('没', 290),
 ('肖战', 288),
 ('发现', 275),
 ('女儿', 275),
 ('月', 275),
 ('日本', 273),
 ('中', 271),
 ('妈妈', 270),
 ('韩国', 258),
 ('孩子', 243),
 ('杨紫', 241),
 ('真的', 236),
 ('买', 236),
 ('去世', 235),
 ('杭州', 234),
 ('公司', 222),
 ('直播', 221),
 ('拍', 220),
 ('做', 212),
 ('2023', 210),
 ('通报', 208),
 ('新', 207),
 ('上海', 203),
 ('警方', 203),
 ('事件', 198),
 ('杨幂', 196),
 ('穿', 196),
 ('视频', 195),
 ('天', 194),
 ('易烊千玺', 192),
 ('学生', 190),
 ('魏大勋', 189),
 ('里', 187),
 ('20', 186),
 ('儿子', 185),
 ('地震', 184),
 ('官宣', 184),
 ('棣', 183),
 ('时', 182),
 ('王一博', 181),
 ('网友', 180),
 ('北京', 179),
 ('女', 179),
 ('TFBOYS', 178),
 ('结婚', 177),
 ('小时', 175),
 ('王鹤', 172),
 ('带', 172),
 ('手机', 172),
 ('曝', 169),
 ('高考', 168),
 ('花', 167),
 ('老人', 167),
 ('名', 166),
 ('男孩', 165),
 ('道歉', 163),
 (

#### 选出出现在热搜50次以上的名字，为了更好的展示这些数据，用Highcharts 进行可视化。

<img src="treemap1.png" align="left" >

#### 以上就是23年度的“热搜之王”

### 3.网民精神状态分析

#### 由于比较好奇23年大家整体的精神状态是如何的，在这里引入一个第三库（snownlp），用来判断文本的情感倾向，返回的是积极性的概率，数值介于0-1之间。

#### 把所有内容拼成一个字符串

In [25]:
from snownlp import SnowNLP

In [26]:
for word in clean_cut_word:
    text1 = ','.join(word) 
s1 = SnowNLP(text1)
print(s1.sentiments)

0.529024453354141


#### 得到的值为0.529，可以看出2023年的微博热搜的用语的情感倾向整体上是中等微微偏上的。？

#### 接下来看看每一条热搜的情况，再将积极的和消极的分别统计出来。

In [27]:
sentiments_each_weibo =[]
for word in title_list:
    temp = SnowNLP(word)
    sentiments_each_weibo.append(temp.sentiments)
sentiments_each_weibo

[0.3212474228189949,
 0.7963504383665548,
 0.6079224996622041,
 0.8047453590635314,
 0.8154089858670225,
 0.6267218540854844,
 0.6431126630364359,
 0.8827177408759362,
 0.5225802334617956,
 0.8480594235241341,
 0.9889940571444652,
 0.7420829345142126,
 0.9527336718642047,
 0.8230573436060115,
 0.2290120385868849,
 0.6369171222969149,
 0.28139513439631236,
 0.8232826951592747,
 0.7297953905324696,
 0.816129547075578,
 0.2200708556446601,
 0.5017187141009336,
 0.6864403227850107,
 0.9752402390648519,
 0.28123186242158194,
 0.9670724356869447,
 0.6549853268015957,
 0.9297612634747083,
 0.5560787996130092,
 0.8069788162625721,
 0.7395201018522584,
 0.5,
 0.2464443062482008,
 0.586511035266812,
 0.301287206266716,
 0.9941109333384169,
 0.34184705760349243,
 0.3106125230865229,
 0.9821338433177484,
 0.5469999639259928,
 0.24386473383421414,
 0.4720602358653613,
 0.421874597530976,
 0.10932685038294054,
 0.9292541936201975,
 0.7844410496596699,
 0.9948721753321005,
 0.3290394254385606,
 0.691

In [28]:
positive_content=[]
negative_content=[]
for y in sentiments_each_weibo:
    if y >= 0.5:
        positive_content.append(y)
    else:
        negative_content.append(y)

In [29]:
print('积极的微博数量为：'+str(len(positive_content)))
print('消极的微博数量为：'+str(len(negative_content)))

积极的微博数量为：25044
消极的微博数量为：12623


#### 整体来看积极的数量是消极的两倍左右，这个数据可以当前社会的主导情绪是相对积极的，从另一个角度看就是媒体在选择热搜话题时，可能更倾向于选择那些具有积极价值、能够引发正面讨论的话题。

#### 接下来分析一下全年微博热搜的情感是如何变化的，先将情感指数、热搜话题、和时间储存起来


In [30]:
df3 = pd.DataFrame(df1['date'].tolist(),columns=['date'])
df3 = pd.concat([df3, pd.DataFrame(sentiments_each_weibo,columns=['sentiments'])],axis=1)
df3 

,date,sentiments
0,20231030,0.321247
1,20230201,0.796350
2,20231219,0.607922
3,20230706,0.804745
4,20230706,0.815409
...,...,...
37662,20230406,0.310416
37663,20231118,0.318506
37664,20230924,0.671403
37665,20231219,0.432878


#### 将每一天微博热搜的情感指数和搜索量分别进行加和，再将情感的数据，与搜索量的数据，放在一个图表中

In [31]:
df4 = df3.groupby("date").sum()
df4

,sentiments
date,
20230101,88.758153
20230102,48.190468
20230103,49.699897
20230104,68.444212
20230105,66.183616
...,...
20231227,58.681539
20231228,60.394334
20231229,59.213627


In [32]:
df5 = df1.drop(['title'],axis=1)
df5

,date,searchCount
31471,20231030,133098325
2922,20230201,71434818
36438,20231219,63844263
17375,20230706,60048669
17393,20230706,45339693
...,...,...
8942,20230406,500039
33326,20231118,500033
27562,20230924,500024
36529,20231219,500019


In [33]:
df6 = df5.groupby("date").sum()
df6

,searchCount
date,
20230101,278035054
20230102,80382361
20230103,91733267
20230104,113412903
20230105,101961616
...,...
20231227,107629885
20231228,103480112
20231229,104835135


In [34]:
import pyecharts.options as opts
from pyecharts.charts import Grid, Line

In [35]:
x_data = df4.index.tolist()
y_data_search = df6['searchCount'].tolist()
y_data_sentiment = df4['sentiments'].tolist()

line1 = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="搜索量",
        y_axis=y_data_search,
        linestyle_opts=opts.LineStyleOpts(width=2),
        label_opts=opts.LabelOpts(is_show=False)
    )

    .set_global_opts(
        title_opts=opts.TitleOpts(title="微博热搜搜索量", pos_left="center"),
        legend_opts=opts.LegendOpts(pos_left="left")
    )
    .set_global_opts(
        datazoom_opts=[opts.DataZoomOpts(range_start=0,range_end=100)])
    .render_notebook()
)
line1

#### 从上图不难看出在1.22、7.6、8.6、8.24、10.27这几日达到了搜索量的高峰，而其他时候都在一定稳定的范围内波动。这些日期引发大量搜索和讨论的事件涉及到各个领域，像1.22当日就是2023年的春晚的下一天，春晚很好看，微博更好玩，春晚的热度次日在微博持续发酵；在7月6日，李玟因去世引发了微博网民的高度关注和深切哀悼，这位知名音乐人和艺术家的离世，在社交媒体上引发了广泛的悼念和回忆；在8月6日发生的地震引发了微博网民的深切关注和祈福，面对自然灾害带来的破坏和伤害，网民们纷纷表达对受灾地区的深切关怀和祈福之情，而8月24日，日本宣布计划排放核废水，这一决定在国际社会引起了广泛关注和担忧，在微博上也引发了强烈的讨论和反对声音；10.27是个悲痛的日子，在这一天，李克强同志逝世的消息传来，引发了广泛的悲伤和哀悼，人们对他的离去深感惋惜，并表示将永远怀念他的崇高品德和卓越领导才能。

### 从这些峰值的内容不难看出，微博网友的关注点主要还是在于人民健康与生命安全、国家外交及政治、以及少量娱乐圈特大事件。虽然单从热搜话题的词云来看，微博网友似乎对娱乐圈的资讯更感兴趣，但能引来大量讨论的，仍然是关乎国家和健康的公众议题。这一结果一改我们对微博娱乐化印象的常态，微博依然肩负媒体的社会监督功能。

In [36]:
line2 = (
    Line()
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="情感指数",
        y_axis=y_data_sentiment,
        linestyle_opts=opts.LineStyleOpts(width=2),
        label_opts=opts.LabelOpts(is_show=False)
    )

    .set_global_opts(
        title_opts=opts.TitleOpts(title="微博热搜情感指数", pos_left="center"),
        legend_opts=opts.LegendOpts(pos_left="left"),
        yaxis_opts=opts.AxisOpts(
            min_=45
        )
    )
    .set_global_opts(
        datazoom_opts=[opts.DataZoomOpts(range_start=0,range_end=100)])
    .render_notebook()
)
line2

#### 根据微博网友关注的热搜内容来观察，我们可以看到在2023年的全年情感指数中，7月和8月的情感指数特别高，而4月份则相对低迷。这种情况可以通过热搜内容的分析来解释。的上升。的上升。

#### 在4月份，负面新闻占据了较大比例，这可能包括一些社会问题、突发事件或者名人绯闻等，这些负面新闻的出现导致了微博网友情感指数的下降。人们可能更多地关注于负面事件的发生，产生了担忧、忧虑或愤怒等负面情绪，从而影响了整体的情感指数。
#### 而在7月和8月份，一方面是因为大型体育赛事如大运会的举办，另一方面是因为大量的演唱会等娱乐活动的举办，这些活动都为微博网友提供了丰富多彩的精神享受和娱乐体验，带来了愉悦、兴奋和自豪等积极的情感体验，从而推动了情感指数的上升。